In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_theme()

penguins = sns.load_dataset('penguins').dropna()
print(penguins.head())


In [ ]:
gentoo = penguins[penguins.species == 'Gentoo'][:50]
adelie = penguins[penguins.species == 'Adelie'][:50]
chinstrap = penguins[penguins.species == 'Chinstrap'][:50]

In [ ]:
plt.scatter(
    chinstrap.bill_length_mm, chinstrap.bill_depth_mm, color="blue", label="chinstrap"
)
plt.scatter(gentoo.bill_length_mm, gentoo.bill_depth_mm, color="magenta", label="gentoo")
plt.xlabel('Bill Length')
plt.ylabel('Bill Depth')
plt.title('Gentoo vs Chinstrap')
plt.legend()

plt.show()

In [ ]:
plt.scatter(
    chinstrap.bill_depth_mm, np.ones(chinstrap.shape[0]), color="blue", label="Chinstrap"
)
plt.scatter(gentoo.bill_depth_mm, np.zeros(gentoo.shape[0]), color="magenta", label="gentoo")
plt.xlabel('Bill Length')
plt.ylabel('Label')
plt.title('Chinstrap vs Gentoo Bill Length')
plt.vlines(17, 0, 1)
plt.legend()

plt.show()

In [ ]:
def sigmoid_preactivation(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
class SingleNeuron(object):
    """
    [WRITE DOCSTRING]
    """
    # TODO: Write docstring
    def __init__(self, activation_function, cost_function):
        self.activation_function = activation_function
        self.cost_function = cost_function

    def train(self, X, y, alpha = 0.005, epochs = 50):
        
        self.w = np.random.randn(X.shape[1])
        self.b = np.random.randn()
        self.errors_ = []
        N = X.shape[0]

        for _ in range(epochs):
            agg_cost = 0
            for x_i, y_i in zip(X, y):
                iter_error = (self.predict(x_i) - y_i)
                self.w -= alpha*iter_error*x_i
                self.b -= alpha*iter_error
                agg_cost += self.cost_function(self.predict(x_i), y_i)
            self.errors_.append(agg_cost/N)
        
        return self

    def predict(self, X):
        preactivation = np.dot(X, self.w) + self.b
        return self.activation_function(preactivation)

    def plot_cost_function(self):
        fig, axs = plt.subplots(figsize = (10, 8))
        axs.plot(range(1, len(self.errors_) + 1), 
                self.errors_,
                label = "Cost function")
        axs.set_xlabel("epochs", fontsize = 15)
        axs.set_ylabel("Cost", fontsize = 15)
        axs.legend(fontsize = 15)
        axs.set_title("Cost Calculated after Epoch During Training", fontsize = 18)
        plt.show()

In [ ]:
def cross_entropy_loss(y_hat, y):
    return - y*np.log(y_hat) - (1 - y)*np.log(1 - y_hat)

In [ ]:
X = gentoo.bill_depth_mm.values
X = np.append(X, chinstrap.bill_depth_mm.values).reshape(-1, 1)
y_temp = np.append(gentoo.species, chinstrap.species)
y = np.where(y_temp == "Gentoo", 0, 1)

In [ ]:
node = SingleNeuron(sigmoid_preactivation, cross_entropy_loss)

In [ ]:
node.train(X, y, alpha = 0.01, epochs = 10000)

In [ ]:
node.plot_cost_function()

In [ ]:
max(chinstrap.bill_depth_mm)
min(gentoo.bill_depth_mm)

In [ ]:
plt.scatter(
    chinstrap.bill_depth_mm, np.ones(chinstrap.shape[0]), color="blue", label="Chinstrap"
)
plt.scatter(gentoo.bill_depth_mm, np.zeros(gentoo.shape[0]), color="magenta", label="Gentoo")
plt.xlabel('Bill Length')
plt.ylabel('Label')
plt.title('Chinstrap vs Adelie Bill Length')
domain = np.linspace(13.1, 20.8, 100)
plt.plot(domain, node.predict(domain.reshape(-1, 1)))
plt.legend()
plt.show()

In [108]:
outcomes = np.rint(node.predict(X)).astype(int)
comparison = outcomes == y
success = sum(comparison.astype(int)) / len(comparison)
error = 1 - success
print("Model Error: ", error)

Model Error:  0.16000000000000003
